# Classifying tissues from gene expression profiles

A problem clinicians often encounter is the primary origin of cancer. Often, metastatic tumors are never fully diagnosed even after various clinical tests to determine the primary site and are thus labeled cancers of unknown origin (CUP).

RNA-seq data seems like a promising solution to the problem, since tissues differ substantially in their gene expression profiles.

## Exercise:

In this exercise, we will learn how to classify a gene expression count sample into its respective tissue. If there are more than two tissues, we call this a multi-class classification problem.

In the first part of the exercise you will have to choose an appropiate loss function for the problem and you will have to decide upon the neural network architecture. In the first 10 minutes of the exercise, discuss the following with your classmates:

1. Which loss function should you use? (Hint: Look at Anders lecture for a recap)
2. Which architecture (number of layers and hidden units) should you use?



After you made yourself familiar with the code and the results from our first trained neural network, try out a few things. Here are some guiding questions:

1.   Does the model overfit?
2.   Try to improve the model by changing model width (number of hidden units in a layer) and model depth (number of hidden layers).
3.   Does the model perform equally well for all tissues?





## The data

### Loading the data

We have prepared a small subset from the Genotype-Tissue Expression (GTEx) database. It contains 3382 samples with expression counts from 1000 genes for 7 different tissues. You can see the raw counts below.

In [ ]:
!wget https://zenodo.org/record/7822717/files/gtex_1000.csv.gz

In [ ]:
import pandas as pd

gtex_raw = pd.read_table("gtex_1000.csv.gz",sep='\t')
gtex_raw

As you can see in the las column of the dataframe, the tissues are enconded as strings, and pytorch does not support this. To address so, we will simply encode the tissues as numbers. Luckily, sklearn has a function for that! See the before and after below.

In [ ]:
# tissues before transform
print('tissues before \n', gtex_raw['tissue'])

# transforming strings to integer labels
gtex = gtex_raw.copy()
#Create a set of unique labels
label_set = set(gtex['tissue'])  
# Create a dictionary mapping labels to numbers
label_dict = {label: i for i, label in enumerate(label_set)}  
# Encode each label using the dictionary
gtex['tissue'] = [label_dict[label] for label in gtex['tissue']]
# tissues after transform
print('tissues after \n', gtex['tissue'])

### An initial exploration of the data using PCA

Before diving "deep" into a neural network, we will explore the data a bit using a principal component analysis (PCA). This will serve as a good baseline to see how easy it is to (linearly) separate the tissues from the data. In this way, we can have an initial guess how complex our model should be.

Due to the extreme range of the expression counts, we log-transform the data before visualizing it in a PCA to get a "clearer visual".

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

# log scale the data
x = gtex.loc[:, gtex.columns.drop('tissue')].values
x = np.log(x+0.01) # add pseudo-counts

# fit pca 
pca = PCA(n_components=2)
principal_components = pca.fit_transform(x)
df_pca = pd.DataFrame(data = principal_components
             , columns = ['principal component 1', 'principal component 2'])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale = 2)

# add the original tissue strings to the PCA data
df_pca['tissue'] = gtex_raw['tissue'].values

fig, ax = plt.subplots(figsize=(10,10))
sns.scatterplot(data=df_pca, x='principal component 1', y='principal component 2', hue='tissue', ax=ax)
ax.legend(bbox_to_anchor=(1.05, 1.), loc='upper left')
plt.show()

From the PCA, it becomes apparent that tissues have different gene expression profiles. Let's try to create a neural network that classifies the samples given the gene expression values.

### Preliminaries: preparing the data for pytorch

Pytorch has certain requirements that datasets need to fulfill. We don't have to bother with the details, though. The Dataset module takes care of things for us, we just need to use it to specify what the input and what the output is, and how to get samples from it.

In more detail, since datasets can be of different types, we need to create new class functions for making the dataset (\__init__\()), telling us how many samples there are (\__len__\()), and accessing samples by index (\__getitem__\()).

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class GeneExpressionDataset(Dataset):
    '''
    Creates a Dataset class for gene expression dataset
    gene_dim is the number of genes (features)
    The rows of the dataframe contain samples, and the 
    columns contain gene expression values 
    and the class label (tissue) at label_position.
    '''
    def __init__(self, gtex, label_position=-1):
        '''
        Args:
            gtex: pandas dataframe containing input and output data
            label_position: column id of the class labels
        '''
        self.dataframe = gtex
        self.label_position = label_position

        self.label = torch.tensor(self.dataframe.iloc[:,label_position].to_numpy())
        self.data = torch.tensor(self.dataframe.drop(self.dataframe.columns[[self.label_position]], axis=1).values).float()
        
    def __len__(self):
        return(len(self.dataframe))
    
    def __getitem__(self, idx):
        # get expression and labels
        expression = self.data[idx,:]
        label = self.label[idx]
        return expression, label

## The neural network

Once we have defined our dataset, we need to define our neural network. 

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=1000,out_features=200)
        self.fc2 = nn.Linear(in_features=200,out_features=7)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Training

Finally, we can move to the main part. We first create a train and test set from the data. This is important for knowing how the model performs on unseen samples from the same "distribution".

We then use these two sets to make so-called dataloaders. These essentially take care of splitting the data into randomly grouped subsets (they are called mini-batches).

After that we initialize the network and define the criterion, aka the loss function, and the optimizer that will update the model weights for us.

In [ ]:
from sklearn.model_selection import train_test_split
from torch import optim

train, test = train_test_split(gtex,test_size=0.2,random_state=42)

gtex_train = GeneExpressionDataset(train)
gtex_test = GeneExpressionDataset(test)
batch_size = 32
train_loader = torch.utils.data.DataLoader(gtex_train, batch_size=batch_size, num_workers=0, shuffle=True)
test_loader = torch.utils.data.DataLoader(gtex_test, batch_size=batch_size, num_workers=0, shuffle=True)

epochs = 20

net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-6)

Now we can finally start training!

In [ ]:
# in these lists we store loss and accuracy per epoch to keep track of the progress
train_loss = []
test_loss = []
train_acc = []
test_acc  = []

for epoch in range(epochs):
    # for each epoch we collect the stats as averages over the mini-batches
    epoch_train_loss = 0.0
    epoch_test_loss = 0.0
    epoch_train_acc = 0.0
    epoch_test_acc = 0.0
    
    # train
    for inputs,labels in train_loader:
        
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + loss
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        epoch_train_loss += loss.item()
        # backward
        loss.backward()
        # optimize
        optimizer.step()

        # calculate train accuracy
        correct_pred = outputs.argmax(dim=1) == labels
        acc = correct_pred.sum() / len(correct_pred)
        epoch_train_acc += acc.item()

    # test
    for inputs,labels in test_loader:

        # forward only
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        epoch_test_loss += loss.item()
        
        correct_pred = outputs.argmax(dim=1) == labels
        acc = correct_pred.sum() / len(correct_pred)
        epoch_test_acc += acc.item()
        
    # average the stats and append to lists
    epoch_train_loss /= len(train_loader)
    epoch_test_loss /= len(test_loader)
    epoch_train_acc /= len(train_loader)
    epoch_test_acc /= len(test_loader)
    train_loss.append(epoch_train_loss)
    test_loss.append(epoch_test_loss)
    train_acc.append(epoch_train_acc)
    test_acc.append(epoch_test_acc)
                                
    print("Epoch:", epoch)
    print("train loss:", round(epoch_train_loss, 4),"test loss:", round(epoch_test_loss, 4))
    print("train acc:", round(epoch_train_acc, 4),"test accuracy:", round(epoch_test_acc, 4))

Lets now plot the loss curves

In [ ]:
plt.plot(np.arange(epoch+1),train_loss,label="Train")
plt.plot(np.arange(epoch+1),test_loss,label="Test")
plt.legend()
plt.xlabel("epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
plt.plot(np.arange(epoch+1),train_acc,label="Train")
plt.plot(np.arange(epoch+1),test_acc,label="Test")
plt.legend()
plt.xlabel("epoch")
plt.ylabel("Accuracy")
plt.show()